# Semantic Kernel с интеграция на OpenBnB MCP сървър

Този тетрадка демонстрира как да използвате Semantic Kernel с реалния OpenBnB MCP сървър за търсене на реални квартири в Airbnb, използвайки MCPStdioPlugin. За достъп до LLM се използва Microsoft Foundry. За да настроите променливите на средата си, можете да следвате [Setup Lesson ](/00-course-setup/README.md)


## Импортиране на необходимите пакети


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Създаване на MCP приставковата връзка

Ще се свържем с [OpenBnB MCP сървъра](https://github.com/openbnb-org/mcp-server-airbnb) чрез MCPStdioPlugin. Този сървър предоставя функционалност за търсене в Airbnb чрез пакета @openbnb/mcp-server-airbnb.


## Създаване на клиента

В този пример ще използваме Microsoft Foundry за достъп до нашия LLM. Уверете се, че вашите системни променливи са настроени правилно.


## Конфигурация на средата

Конфигурирайте настройките на Azure OpenAI. Уверете се, че имате зададени следните променливи на средата:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Разбиране на интеграцията с OpenBnB MCP

Този ноутбук се свързва с **реалния OpenBnB MCP сървър**, който предоставя реална функционалност за търсене в Airbnb.

### Как работи:

1. **MCPStdioPlugin**: Използва стандартна входно/изходна комуникация със MCP сървъра
2. **Истински NPM пакет**: Изтегля и изпълнява `@openbnb/mcp-server-airbnb` чрез npx
3. **Живи данни**: Връща реални данни за имоти от Airbnb чрез техните API-та
4. **Откриване на функции**: Агентът автоматично открива наличните функции от MCP сървъра

### Налични функции:

Обикновено OpenBnB MCP сървърът предоставя:
- **search_listings** - Търсене на имоти в Airbnb по локация и критерии
- **get_listing_details** - Вземане на подробна информация за конкретни имоти
- **check_availability** - Проверка на наличността за конкретни дати
- **get_reviews** - Получаване на ревюта за имоти
- **get_host_info** - Вземане на информация за домакините на имотите

### Предварителни изисквания:

- **Node.js** инсталиран на вашата система
- **Интернет връзка** за изтегляне на MCP сървърния пакет
- **NPX** наличен (идва с Node.js)

### Тестване на връзката:

Можете да тествате MCP сървъра ръчно, като изпълните:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Това ще изтегли и стартира OpenBnB MCP сървъра, към който после Semantic Kernel се свързва за реални данни от Airbnb.


## Стартиране на агента с OpenBnB MCP сървъра

Сега ще стартираме AI агента, който се свързва със сървъра OpenBnB MCP, за да търси реални помещения в Airbnb в Стокхолм за 2 възрастни и 1 дете. Чувствайте се свободни да промените списъка `user_inputs`, за да модифицирате критериите за търсене.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Обобщение
Поздравления! Вие успешно създадохте AI агент, който се интегрира с реално търсене на настаняване, използвайки протокола Model Context Protocol (MCP):

## Използвани технологии:
- Semantic Kernel - За изграждане на интелигентни агенти с Azure OpenAI
- Microsoft Foundry - За възможности на големи езикови модели и чат завършване
- MCP (Model Context Protocol) - За стандартизирана интеграция на инструменти
- OpenBnB MCP Server - За реална функционалност на търсене в Airbnb
- Node.js/NPX - За стартиране на външния MCP сървър

## Какво научихте:
- MCP интеграция: Свързване на Semantic Kernel агенти с външни MCP сървъри
- Достъп до данни в реално време: Търсене на реални имоти в Airbnb чрез живи API-та
- Протоколната комуникация: Използване на stdio комуникация между агента и MCP сървъра
- Откриване на функции: Автоматично откриване на налични функции от MCP сървъри
- Поточно отразяване: Заснемане и записване на извиквания на функции в реално време
- Рендиране на HTML: Форматиране на отговорите на агента с елегантни таблици и интерактивни елементи

## Следващи стъпки:
- Интегриране на допълнителни MCP сървъри (време, полети, ресторанти)
- Създаване на мултиагентна система, комбинираща MCP и A2A протоколи
- Създаване на персонализирани MCP сървъри за ваши собствени източници на данни
- Имплементиране на постоянна памет за разговори между сесиите
- Разгръщане на агента в Azure Functions с оркестрация на MCP сървъра
- Добавяне на потребителска автентикация и възможности за резервации


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:
Този документ е преведен с помощта на AI преводачески услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или неправилни тълкувания, възникнали в резултат на използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
